In [1]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [3]:
cd gdrive/MyDrive

/content/gdrive/MyDrive


In [4]:
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval


class ElementDiscoveryEval(COCOeval):
    def __init__(self, coco_gt, coco_dt):
        super(ElementDiscoveryEval, self).__init__(coco_gt, coco_dt, "bbox")

# Load results from tsv file, provide ground truth id and cluster index headers. Defaults: 'id', 'cluster'.
gt = "/content/gdrive/MyDrive/SAGPACE/coco_format_gt.json"
dt = "/content/gdrive/MyDrive/SAGPACE/detections_tune-on-real_new.json"

# Read the JSON file containing the annotations
with open(gt, 'r') as f:
    annotations = json.load(f)

# Convert annotations to COCO format
coco_annotations = {
    "images": [],
    "annotations": [],
    "categories": []
}

image_id_map = {}
category_id_map = {}

for annotation in annotations:
    # Assuming each annotation contains 'image_id', 'category_id', 'bbox', and 'score'
    image_id = annotation['image_id']
    category_id = annotation['category_id']
    bbox = annotation['bbox']

    if image_id not in image_id_map:
        # Add image to image map if not already present
        image_id_map[image_id] = len(image_id_map) + 1
        coco_annotations['images'].append({"id": image_id_map[image_id], "file_name": str(image_id) + ".jpg"})

    # Add category to category map if not already present
    if category_id not in category_id_map:
        category_id_map[category_id] = len(category_id_map) + 1
        coco_annotations['categories'].append({"id": category_id_map[category_id], "name": str(category_id)})

    coco_annotations['annotations'].append({
        "id": len(coco_annotations['annotations']) + 1,
        "image_id": image_id_map[image_id],
        "category_id": category_id_map[category_id],
        "bbox": bbox,
        "iscrowd": 0,
        'area': bbox[3] * bbox[2]
    })

# Create COCO instances
coco_gt = COCO()
coco_gt.dataset = coco_annotations
coco_gt.createIndex()

# Read the JSON file containing the annotations
with open(dt, 'r') as f:
    annotations = json.load(f)

# Convert annotations to COCO format
coco_annotations = {
    "images": [],
    "annotations": [],
    "categories": []
}

image_id_map = {}
category_id_map = {}

# Check if data is a dictionary
#print_keys(annotations)
for annotation in annotations:
    # Assuming each annotation contains 'image_id', 'category_id', 'bbox', and 'score'
    image_id = annotation['img']
    for anns in annotation['anns']:
        category_id = anns['id']
        bbox = anns['bbox']
        score =  anns['score']

        if image_id not in image_id_map:
          # Add image to image map if not already present
          image_id_map[image_id] = len(image_id_map) + 1
          coco_annotations['images'].append({"id": image_id_map[image_id], "file_name": str(image_id) + ".jpg"})

        # Add category to category map if not already present
        if category_id not in category_id_map:
          category_id_map[category_id] = len(category_id_map) + 1
          coco_annotations['categories'].append({"id": category_id_map[category_id], "name": str(category_id)})

        coco_annotations['annotations'].append({
          "id": len(coco_annotations['annotations']) + 1,
          "image_id": image_id_map[image_id],
          "category_id": category_id_map[category_id],
          "bbox": bbox,
          "iscrowd": 0,
          'area': bbox[3] * bbox[2],
          'score':score
        })

# Create COCO instances
coco_dt = COCO()
coco_dt.dataset = coco_annotations
coco_dt.createIndex()

# Example usage of COCO API
# Get all category IDs
category_ids = coco_gt.getCatIds()

# Get all image IDs
image_ids = coco_gt.getImgIds()

# Get annotations for a specific image
image_id = image_ids[0]
annotations_ids = coco_gt.getAnnIds(imgIds=image_id)
annotations = coco_gt.loadAnns(annotations_ids)
print("Annotations for Image {}: {}".format(image_id, annotations))
annotations = coco_dt.loadAnns(annotations_ids)
print("Annotations for Image {}: {}".format(image_id, annotations))



creating index...
index created!
creating index...
index created!
Annotations for Image 1: [{'id': 1, 'image_id': 1, 'category_id': 1, 'bbox': [28.0, 26.0, 111.0, 74.0], 'iscrowd': 0, 'area': 8214.0}, {'id': 2, 'image_id': 1, 'category_id': 2, 'bbox': [26.0, 100.0, 111.0, 167.0], 'iscrowd': 0, 'area': 18537.0}, {'id': 3, 'image_id': 1, 'category_id': 2, 'bbox': [27.0, 270.0, 114.0, 163.0], 'iscrowd': 0, 'area': 18582.0}, {'id': 4, 'image_id': 1, 'category_id': 1, 'bbox': [31.0, 432.0, 109.0, 75.0], 'iscrowd': 0, 'area': 8175.0}, {'id': 5, 'image_id': 1, 'category_id': 3, 'bbox': [179.0, 81.0, 77.0, 70.0], 'iscrowd': 0, 'area': 5390.0}, {'id': 6, 'image_id': 1, 'category_id': 4, 'bbox': [155.0, 152.0, 97.0, 68.0], 'iscrowd': 0, 'area': 6596.0}, {'id': 7, 'image_id': 1, 'category_id': 5, 'bbox': [155.0, 221.0, 101.0, 111.0], 'iscrowd': 0, 'area': 11211.0}, {'id': 8, 'image_id': 1, 'category_id': 4, 'bbox': [158.0, 333.0, 99.0, 69.0], 'iscrowd': 0, 'area': 6831.0}, {'id': 9, 'image_id': 1

In [5]:
def print_keys(data, indent=''):
    if isinstance(data, dict):
        for key, value in data.items():
            print(indent + key)
            if isinstance(value, dict):
                print_keys(value, indent + '  ')
            elif isinstance(value, list):
                for item in value:
                    print_keys(item, indent + '  ')
    elif isinstance(data, list):
        for item in data:
            print_keys(item, indent + '  ')

In [6]:
# Run evaluation
# Initialize evaluation class
fleurons_eval = ElementDiscoveryEval(coco_dt=coco_dt, coco_gt=coco_gt)
fleurons_eval.evaluate()

#Print summary
fleurons_eval.accumulate()
fleurons_eval.summarize()


Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.48s).
Accumulating evaluation results...
DONE (t=0.33s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100